Scraping data from Wikipedia list.

In [4]:
import pandas as pd

 
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)[0]
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing postcodes with unassigned boroughs and combining rows with a shared postcode.

In [5]:
tables = tables[tables['Borough'] != "Not assigned"]
tables = tables.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [6]:
tables.shape

(103, 3)

Part Two

In [7]:
geo = pd.read_csv("http://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Merge data into one dataframe
df_merged = pd.merge(tables, geo, left_on='Postcode', right_on='Postal Code')

In [9]:
#Drop redundant postcode column
df_merged = df_merged.drop(['Postal Code'], axis=1)
df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Part Three

In [10]:
import numpy as np
from sklearn.cluster import KMeans
import folium
print('Installed and imported')

Installed and imported


In [14]:
kclusters = 8
df_cluster = df_merged.drop('Borough',axis=1)
df_cluster = df_cluster.drop('Postcode',axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster.drop('Neighbourhood',1))
kmeans.labels_[0:10]

array([5, 5, 5, 5, 5, 5, 1, 1, 1, 1], dtype=int32)

In [15]:
df_merged.insert(0,'Cluster Labels',kmeans.labels_)

In [16]:
df_merged.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,5,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,5,M1G,Scarborough,Woburn,43.770992,-79.216917
4,5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

#This map code is based on that from the Lab session.
map_clusters = folium.Map(location=[43.723056, -79.397963], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters